In [15]:
import torch
import pandas as pd
import numpy as np
import os
import tqdm
import pytorch_lightning as pl
from torchsummary import summary
from matplotlib import pyplot as plt
from pytorch_lightning.core.lightning import LightningModule
from torch import nn

In [4]:
import torchvision
torchvision.__version__

'0.11.3'

In [5]:
print(torch.__version__)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
print(torch.cuda.get_device_name(0))

1.10.2
cuda
NVIDIA GeForce MX250


In [6]:
import torchmetrics

In [39]:
from torch.optim import Adam

In [70]:
class model(LightningModule):
    def __init__(self):
        super().__init__()
        self.loss_fn = nn.CrossEntropyLoss().to(device)
        # self.metrics = torchmetrics.functional.accuracy()
        self.net = nn.Sequential(
            nn.Linear(32, 16),
            nn.BatchNorm1d(16),
            nn.ReLU(inplace=True),

            nn.Linear(16, 8),
            nn.BatchNorm1d(8),
            nn.ReLU(inplace=True),
            
            nn.Linear(8, 4)
        ).to(device)

    def forawrd(self, x):
        return self.net(x)
    
    def training_step(self, batch, batch_idx):
        x = batch[:, :32] 
        target = batch[:, 32].to(torch.int64)
        pred = self.forawrd(x)
        loss = self.loss_fn(pred, target)
        acc = torchmetrics.functional.accuracy(pred, target)

        self.log("train_acc", acc, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        self.log("train_loss", loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        return {"loss" : loss, "acc" : acc}
    
    # def validation_step(self, batch, batch_idx):
    #     x, target = batch
    #     pred = self.net(x)
    #     loss = self.loss_fn(pred, target)
    #     acc = torchmetrics.functional.accuracy()(pred, target)
        
    #     self.log("valid_acc", acc, on_step=True, on_epoch=True, prog_bar=True, logger=True)
    #     self.log("valid_loss", loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
    #     {"loss" : loss, "acc" : acc}

    def configure_optimizers(self):
        return Adam(self.parameters(), lr=0.002)

In [54]:
def create_datasets(batch_size):
    valid_size = 0.2

    # torch.FloatTensor로 변환
    train_csv = pd.read_csv('../data/train.csv')
    test_csv = pd.read_csv('../data/test.csv')

    train_data = torch.tensor(train_csv.to_numpy()[:, 1:]).float()
    test_data = torch.tensor(test_csv.to_numpy()[:, 1:]).float()

    # validation으로 사용할 trainning indices를 얻는다.
    num_train = len(train_data)
    indices = list(range(num_train))
    np.random.shuffle(indices)
    split = np.int32(np.floor(valid_size * num_train))
    train_idx, valid_idx = indices[split:], indices[:split]

    vaild_data = train_data[valid_idx]
    train_data = train_data[train_idx]
    # train_sampler = SubsetRandomSampler(train_idx)
    # valid_sampler = SubsetRandomSampler(valid_idx)

    # load training data in batches
    train_loader = torch.utils.data.DataLoader(train_data,
                                               batch_size=batch_size,
                                            #    sampler=train_sampler,
                                               num_workers=0)

    # load validation data in batches
    valid_loader = torch.utils.data.DataLoader(train_data,
                                               batch_size=batch_size,
                                            #    sampler=valid_sampler,
                                               num_workers=0)

    # load test data in batches
    test_loader = torch.utils.data.DataLoader(test_data,
                                              batch_size=batch_size,
                                              num_workers=0)

    return train_loader, test_loader, valid_loader


In [55]:
batch_size = 64

train_loader, test_loader, valid_loader = create_datasets(batch_size)

In [71]:
model_nn = model()
n_epochs = 50

# training
trainer = pl.Trainer(max_epochs=n_epochs, gpus=1)
trainer.fit(model_nn, train_loader)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type             | Params
---------------------------------------------
0 | loss_fn | CrossEntropyLoss | 0     
1 | net     | Sequential       | 748   
---------------------------------------------
748       Trainable params
0         Non-trainable params
748       Total params
0.003     Total estimated model params size (MB)
C:\Users\ktash\.conda\envs\pytorch\lib\site-packages\pytorch_lightning\trainer\data_loading.py:132: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
C:\Users\ktash\.conda\envs\pytorch\lib\site-packages\pytorch_lightning\trainer\data_loading.py:432: UserWarning: The number o

Epoch 49: 100%|██████████| 30/30 [00:00<00:00, 34.66it/s, loss=0.369, v_num=8, train_acc_step=1.000, train_loss_step=0.0905, train_acc_epoch=0.851, train_loss_epoch=0.404]
